In [ ]:
import os
os.chdir('../')
os.getcwd()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from src.io import dataloader

In [ ]:
def preproccess(df, personenzug=False, clean_columns=False, remove_duplicates=False):

    # drop all rows where kind is Personenzug
    if personenzug:
        df = df[~df.kind.isin(['IC-A (12.1)', 'ICE-T (14.1)', 'EC (12.1)', 'RB-D (41.1)', 'RE-D (40.1)', 'IC (12.1)', 'RB (41.1)', 'RE (40.1)',])]

    # delete columns: timetablePeriodID, remarks, trainsetLength, trainsetWeight, compID, sectionID
    if clean_columns:
        df = df.drop(columns=['timetablePeriodID', 'remarks', 'trainsetLength', 'trainsetWeight, compID, sectionID'])

    if remove_duplicates:
        df = df.drop_duplicates(subset='Betriebsstelle', keep='first')
    return df

In [ ]:
def change_to_time(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name], format='%H:%M:%S.00')
    return df

In [ ]:
df_kss = dataloader.load_kss()
df_kss

In [ ]:
df_spurplanbetriebsstellen = dataloader.load_spurplanbetriebsstellen()
df_spurplanbetriebsstellen

In [ ]:
df_spurplanbetriebsstellen = preproccess(remove_duplicates= True, df=df_spurplanbetriebsstellen)
df_spurplanbetriebsstellen

In [ ]:
# df_kss = df_kss.join(other=df_spurplanbetriebsstellen, )
# join df_kss and df_spurplanbetriebsstellen on Betriebsstelle and posID
df_joined = df_kss.merge(right=df_spurplanbetriebsstellen, how='left', left_on='posID', right_on='Betriebsstelle')
df_joined = change_to_time(df_joined, 'publishedDeparture')
df_joined = change_to_float(df_joined, 'X')
df_joined = change_to_float(df_joined, 'Y')
df_joined




In [ ]:
#Shift joined dataframe by
df_joined[['previous_train_id', 'pevious_departure', 'previous_x', 'previous_y']] = df_joined[['trainID', 'publishedDeparture', 'X', 'Y']].shift(1)
df_joined

In [ ]:
df_joined['time_diff'] = df_joined['publishedDeparture']-df_joined['pevious_departure']
df_joined['time_diff'] = df_joined['time_diff'].dt.total_seconds()
# use absolute values of X and Y to calculate distance
df_joined['distance'] = np.sqrt((df_joined['X']-df_joined['previous_x'])**2+(df_joined['Y']-df_joined['previous_y'])**2)
df_joined['speed'] = df_joined['distance']/df_joined['time_diff']

df_joined


In [ ]:
# when trainID changes, set last_speed to 0
df_joined.loc[df_joined['trainID'] != df_joined['previous_train_id'], 'speed'] = 0
# when trainID changes, set distance to 0
df_joined.loc[df_joined['trainID'] != df_joined['previous_train_id'], 'distance'] = 0
df_joined


In [ ]:
sns.histplot(data=df_joined, x='speed', bins=100)


In [ ]:
# pearson correlation of average speed per tranID and total distance
df_joined.groupby('trainID').agg({'speed': 'mean', 'distance': 'sum'}).corr(method='pearson')

In [ ]:
df_joined.dtypes

In [ ]:
df_joined.groupby('Betriebsstelle').agg({'speed': 'mean', 'distance': 'sum'}).sort_values(by='speed', ascending=False)

In [ ]:
df_joined.groupby('trainID').agg({'speed': 'mean', 'totalWeight': 'mean'}).corr(method='pearson')